In [1]:
import syft as sy
import copy

In [2]:
duet1 = sy.join_duet("d65b5ca1d2319946bf05088277bffe1b")
sy.logger.add(sink="./syft_ds.log")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 3063b4e283cf2f1f2976c6c10ad69033

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


In [3]:
duet2 = sy.join_duet("90f5a918a6706ecc9e579b9a6c130f90")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 38e56b1696f7651cef222d3777b229f0

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


In [4]:
import torch
import torchvision

In [5]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 302ab3e42798406aa8e3c8c7cdca4a2a>,[x_train],Dataset of 30000 samples,<class 'torch.Tensor'>
1,<UID: 3191f50488ea46d6b4377a1de71e0dbb>,[y_train],Dataset of 30000 samples,<class 'torch.Tensor'>
2,<UID: 67adb114b96b4a67ac9d6f4e729f0ec1>,[x_test],Dataset of 5000 samples,<class 'torch.Tensor'>
3,<UID: b0611b7a9c1c4c93b13253ab5e1cc5fe>,[y_test],Dataset of 5000 samples,<class 'torch.Tensor'>


In [6]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: a0f0c6a89a264a64974f84ba1574f980>,[x_train],Dataset of 30000 samples,<class 'torch.Tensor'>
1,<UID: ce8affff3db3466389b2dd8617f333d9>,[y_train],Dataset of 30000 samples,<class 'torch.Tensor'>
2,<UID: 5f531de3a296488fbcd0968bda16879f>,[x_test],Dataset of 5000 samples,<class 'torch.Tensor'>
3,<UID: 9cfef27ded0343fba8c3c1a7dc8f34e8>,[y_test],Dataset of 5000 samples,<class 'torch.Tensor'>


In [7]:
#cnn 모델을 사용
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.conv1 = self.torch_ref.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = self.torch_ref.nn.Conv2d(32, 64, 3, 1) 
        self.conv3 = self.torch_ref.nn.Conv2d(64, 128, 3 ,1)
        
        self.dropout1 = self.torch_ref.nn.Dropout2d(0.25)
        self.dropout2 = self.torch_ref.nn.Dropout2d(0.5)
        
        self.fc1 = self.torch_ref.nn.Linear(9216, 128)
        self.fc2 = self.torch_ref.nn.Linear(128, 10)

    def forward(self, x):
        # ImgIn shape=(?, 1, 28, 28)
        # Conv     -> (?, 32, 28, 28)
        x = self.conv1(x)
        x = self.torch_ref.nn.functional.relu(x)
        # ImgIn shape=(?, 32, 28, 28)
        # Conv     -> (?, 64, 28, 28)
        # Pool     -> (?, 64, 14, 14)
        x = self.conv2(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.torch_ref.nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        # Flatten     -> (?, 64*14*14)
        x = self.torch_ref.flatten(x, 1)
        x = self.fc1(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = self.torch_ref.nn.functional.log_softmax(x, dim=1)
        return output

In [8]:
# 학습에 사용될 인자들 설정
args = {
    "images": 60000,
    "clients": 2,
    "rounds": 4,
    "batch_size": 64,
    "test_batch_size": 5000,
    "epochs": 4,
    "lr": 1.0,
    "gamma": 0.7,
    "dry_run": False,
    "seed": 42, # the meaning of life
    "log_interval": 10,
    "torch_seed": 0,
    "save_model": True,
}

In [9]:
clients = []

for i in range(args['clients']):
    clients.append({'duet': eval("duet{}".format(i+1))})

In [10]:
torch.manual_seed(args['torch_seed'])
local_model = SyNet(torch)

In [11]:
#train 데이터셋 얻기
train_kwargs = {
    "batch_size": args["batch_size"],
}
print(len(duet1.store[0]), len(duet1.store[1]))
train_data_ptr1=torch.utils.data.TensorDataset(duet1.store[0].get_copy(), duet1.store[1].get_copy())
clients[0]['train_data_ptr'] = train_data_ptr1
clients[0]['train_loader_ptr'] = torch.utils.data.DataLoader(train_data_ptr1,**train_kwargs)

print(len(duet2.store[0]), len(duet2.store[1]))
train_data_ptr2=torch.utils.data.TensorDataset(duet2.store[0].get_copy(), duet2.store[1].get_copy())
clients[1]['train_data_ptr'] = train_data_ptr2
clients[1]['train_loader_ptr'] = torch.utils.data.DataLoader(train_data_ptr2,**train_kwargs)

30000 30000
30000 30000


In [12]:
#test데이터셋 얻기
test_kwargs = {
    "batch_size": args["test_batch_size"],
}
#duet에 2번째, 3번째가 x_test, y_test
print(len(duet1.store[2]), len(duet1.store[3]))
test_data_ptr1=torch.utils.data.TensorDataset(duet1.store[2].get_copy(), duet1.store[3].get_copy())
clients[0]['test_data_ptr'] = test_data_ptr1
clients[0]['test_loader_ptr'] = torch.utils.data.DataLoader(test_data_ptr1,**test_kwargs)

print(len(duet2.store[2]), len(duet2.store[3]))
test_data_ptr2=torch.utils.data.TensorDataset(duet2.store[2].get_copy(), duet2.store[3].get_copy())
clients[1]['test_data_ptr'] = test_data_ptr2
clients[1]['test_loader_ptr'] = torch.utils.data.DataLoader(test_data_ptr2,**train_kwargs)

5000 5000
5000 5000


In [13]:
clients

[{'duet': <Duet: <UID: ae42d5285b3c464aa952f0bed4aaf1b5>>,
  'train_data_ptr': <torch.utils.data.dataset.TensorDataset at 0x7fbf1a86b880>,
  'train_loader_ptr': <torch.utils.data.dataloader.DataLoader at 0x7fbf1aafd490>,
  'test_data_ptr': <torch.utils.data.dataset.TensorDataset at 0x7fbf1a4a24c0>,
  'test_loader_ptr': <torch.utils.data.dataloader.DataLoader at 0x7fbf1a865c10>},
 {'duet': <Duet: <UID: bcb0f6ce100c41b8b02061b5c8c0f365>>,
  'train_data_ptr': <torch.utils.data.dataset.TensorDataset at 0x7fbf1a3e6cd0>,
  'train_loader_ptr': <torch.utils.data.dataloader.DataLoader at 0x7fbf1a3e6fd0>,
  'test_data_ptr': <torch.utils.data.dataset.TensorDataset at 0x7fbf1ab02f40>,
  'test_loader_ptr': <torch.utils.data.dataloader.DataLoader at 0x7fbf1b833430>}]

In [14]:
for idx, client in enumerate(clients):
    client['remote_torch'] = client['duet'].torch
    torch.manual_seed(idx)
    client['model'] = SyNet(torch)

In [16]:
def train(client, epoch, args):
    # + 0.5 lets us math.ceil without the import
    train_batches = round((client['train_data_length'] / args["batch_size"]) + 0.5)
    print(f"> Running train in {train_batches} batches")
    if client['remote_model'].is_local:
        print("Training requires remote model")
        return

    client['remote_model'].train()

    for batch_idx, data in enumerate(client['train_loader_ptr']):
        data_ptr, target_ptr = data[0], data[1]
        client['optim'].zero_grad()
        output = client['remote_model'](data_ptr)
        loss = client['remote_torch'].nn.functional.nll_loss(output, target_ptr)
        loss.backward()
        client['optim'].step()
        loss_item = loss.item()
        train_loss = client['duet'].python.Float(0)  # create a remote Float we can use for summation
        train_loss += loss_item
        if batch_idx % args["log_interval"] == 0:
            local_loss = None
            local_loss = loss_item.get(
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=5
            )
            if local_loss is not None:
                print("Train Epoch: {} {} {:.4}".format(epoch, batch_idx, local_loss))
            else:
                print("Train Epoch: {} {} ?".format(epoch, batch_idx))
            if args["dry_run"]:
                break
        if batch_idx >= train_batches - 1:
            print("batch_idx >= train_batches, breaking")
            break

In [17]:
def test_local(client, model, test_loader, test_data_length):
    current_model = None
    # download remote model
    if not model.is_local:
        current_model = client['remote_model'].get(
            request_block=True,
            reason="test evaluation",
            timeout_secs=5
        )
    else:
        current_model = model
    # + 0.5 lets us math.ceil without the import
    test_batches = round((test_data_length / args["test_batch_size"]) + 0.5)
    print(f"> Running test_local in {test_batches} batches")
    current_model.eval()
    test_loss = 0.0
    correct = 0.0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            output = current_model(data)
            iter_loss = torch.nn.functional.nll_loss(output, target, reduction="sum").item()
            test_loss = test_loss + iter_loss
            pred = output.argmax(dim=1)
            total = pred.eq(target).sum().item()
            correct += total
            if args["dry_run"]:
                break
                
            if batch_idx >= test_batches - 1:
                print("batch_idx >= test_batches, breaking")
                break

    accuracy = correct / test_data_length
    print(f"Test Set Accuracy: {100 * accuracy}%")

In [18]:
def averageModels(global_model, clients):
    client_models = [clients[i]['model'] for i in range(len(clients))]
    samples = [clients[i]['samples'] for i in range(len(clients))]
    global_dict = global_model.state_dict()
    
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() * samples[i] for i in range(len(client_models))], 0).sum(0)
            
    global_model.load_state_dict(global_dict)
    return global_model

In [19]:
# normally we would not necessarily know the length of a remote dataset so lets ask for it
# so we can pass that to our training loop and know when to stop
def get_train_length(train_data_ptr):
    train_data_length = len(train_data_ptr)
    return train_data_length


for client in clients:
    client['train_data_length'] = get_train_length(client['train_data_ptr'])
    try:
        if train_data_length is None:
            train_data_length = get_train_length(client['train_data_ptr'])
    except NameError:
        train_data_length = get_train_length(client['train_data_ptr'])
    
    client['samples'] = client['train_data_length'] / args['images']
    print(f"Training Dataset size is: {client['train_data_length']}")
    
def get_test_length(test_data_ptr):
    test_data_length = len(test_data_ptr)
    return test_data_length


for client in clients:
    client['test_data_length'] = get_test_length(client['test_data_ptr'])
    try:
        if test_data_length is None:
            test_data_length = get_test_length(client['test_data_ptr'])
    except NameError:
        test_data_length = get_test_length(client['test_data_ptr'])
    
    client['samples'] = client['test_data_length'] / args['images']
    print(f"Test Dataset size is: {client['test_data_length']}")

Training Dataset size is: 30000
Training Dataset size is: 30000
Test Dataset size is: 5000
Test Dataset size is: 5000


In [21]:
%%time
import time
args["dry_run"] = False  # comment to do a full train
print("Starting Training")

for fed_round in range(args['rounds']):
    for i, client in enumerate(clients):
        print("Round Number:",fed_round+1)
        client['remote_model'] = client['model'].send(client['duet']).cpu()
        client['optim'] = client['remote_torch'].optim.Adadelta(client['remote_model'].parameters(), lr=args['lr'])
        client['sched'] = client['remote_torch'].optim.lr_scheduler.StepLR(client['optim'], step_size=1, gamma=args['gamma'])
        
        # train the clients\
        print("client Number:",i)
        for epoch in range(1, args["epochs"] + 1):
            epoch_start = time.time()
            print(f"Epoch: {epoch}")
            # remote training on model with remote_torch
            train(client, epoch, args)
            client['sched'].step()
            epoch_end = time.time()
            print(f"Epoch time: {int(epoch_end - epoch_start)} seconds")
            break
        
        # get the client model back for averaging
        client['model'] = client['remote_model'].get(
            request_block=True,
            reason="test evaluation",
            timeout_secs=5
        )

    # Average all the clients
    local_model = averageModels(local_model, clients)
    
    # local testing on model with local torch
    test_local(client, client['model'], client['test_loader_ptr'], client['test_data_length'])
    
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(copy.deepcopy(local_model.state_dict()))
    
print("Finished Training")

Starting Training
Round Number: 1
client Number: 0
Epoch: 1
> Running train in 469 batches
Train Epoch: 1 0 0.1347
Train Epoch: 1 10 0.3326
Train Epoch: 1 20 0.2078
Train Epoch: 1 30 0.1253
Train Epoch: 1 40 0.08132
Train Epoch: 1 50 0.1343
Train Epoch: 1 60 0.1576
Train Epoch: 1 70 0.07147
Train Epoch: 1 80 0.09125
Train Epoch: 1 90 0.2886
Train Epoch: 1 100 0.1592
Train Epoch: 1 110 0.0834
Train Epoch: 1 120 0.06519
Train Epoch: 1 130 0.07956
Train Epoch: 1 140 0.2916
Train Epoch: 1 150 0.08925
Train Epoch: 1 160 0.07734
Train Epoch: 1 170 0.0881
Train Epoch: 1 180 0.06535
Train Epoch: 1 190 0.05161
Train Epoch: 1 200 0.3112
Train Epoch: 1 210 0.02376
Train Epoch: 1 220 0.0351
Train Epoch: 1 230 0.08015
Train Epoch: 1 240 0.09655
Train Epoch: 1 250 0.1236
Train Epoch: 1 260 0.2171
Train Epoch: 1 270 0.1569
Train Epoch: 1 280 0.1454
Train Epoch: 1 290 0.08106
Train Epoch: 1 300 0.1014
Train Epoch: 1 310 0.07516
Train Epoch: 1 320 0.1219
Train Epoch: 1 330 0.1637
Train Epoch: 1 340 0.1

> Running test_local in 2 batches
batch_idx >= test_batches, breaking
Test Set Accuracy: 2.54%
Round Number: 4
client Number: 0
Epoch: 1
> Running train in 469 batches
Train Epoch: 1 0 2.292
Train Epoch: 1 10 1.775
Train Epoch: 1 20 0.8218
Train Epoch: 1 30 0.4373
Train Epoch: 1 40 0.4484
Train Epoch: 1 50 0.403
Train Epoch: 1 60 0.3707
Train Epoch: 1 70 0.2017
Train Epoch: 1 80 0.1687
Train Epoch: 1 90 0.6354
Train Epoch: 1 100 0.2367
Train Epoch: 1 110 0.1892
Train Epoch: 1 120 0.2775
Train Epoch: 1 130 0.4715
Train Epoch: 1 140 0.3589
Train Epoch: 1 150 0.1605
Train Epoch: 1 160 0.4568
Train Epoch: 1 170 0.1669
Train Epoch: 1 180 0.2684
Train Epoch: 1 190 0.05584
Train Epoch: 1 200 0.3822
Train Epoch: 1 210 0.1409
Train Epoch: 1 220 0.06109
Train Epoch: 1 230 0.0504
Train Epoch: 1 240 0.3342
Train Epoch: 1 250 0.1915
Train Epoch: 1 260 0.2127
Train Epoch: 1 270 0.2771
Train Epoch: 1 280 0.1933
Train Epoch: 1 290 0.2085
Train Epoch: 1 300 0.1011
Train Epoch: 1 310 0.1136
Train Epoch: